In [1]:
import wave
import numpy as np
import matplotlib.pyplot as plt
import struct
import sys
import random
from scipy.io import wavfile

In [2]:
file_path = 'cover_audio.wav'
rate, frame_bytes = wavfile.read(file_path)

In [3]:
frame_bytes = frame_bytes.T
frame_len = frame_bytes.shape[1]

In [4]:
key = input("Enter the secret key: ")
msg = input("Enter the message to hide: ")
msg = msg + "$"
msgInBinary = ''.join([format(ord(i), "08b") for i in msg])
msgLen = len(msgInBinary) + 16
msgLen = format(msgLen, "16b")
msgInBinary = msgLen.replace(' ', '0') + msgInBinary
a = int(key[:len(key)//2]) % ( frame_len // 2)
step = int(key[len(key)//2:])
random.seed(step)
if frame_len < len(msgInBinary):
  print("Message too long")
  sys.exit(0)
range = frame_len // len(msgInBinary)
print(range, step, msgLen.replace(' ', '0'), int(msgLen, 2))
step = random.randint(1,range)

15888 34 0000000001110000 112


In [5]:
strIdx = 0
i = a
while strIdx < len(msgInBinary):
  if i >= frame_len:
    break
  frame_bytes[0][i] = int(bin(frame_bytes[0][i])[:-1] + msgInBinary[strIdx], 2)
  strIdx += 1
  if strIdx < 16:
    i += 1
  else:
    i += step
    random.seed(step)
    step = random.randint(1,range)
if strIdx < len(msgInBinary):
  print("Message too long to hide in audio")
else:
  wavfile.write("stego_audio.wav", rate, frame_bytes.T)
  print("Message hidden successfully")

Message hidden successfully


In [6]:
rate, secret_file = wavfile.read("stego_audio.wav")
secret_bytes = secret_file.T
secret_len = len(secret_bytes[0])


In [7]:
key = "1234"

In [8]:
# key = input("key: ")
ex_msg = ""
temp_msg = ""
i = int(key[:len(key)//2]) % ( secret_len // 2)
msgLen = ""
step = int(key[len(key)//2:])
random.seed(step)
while i < secret_len:
  msgLen += str(secret_bytes[0][i] & 1)
  if len(msgLen) == 16:
    break
  i += 1
range = secret_len // int(msgLen, 2)
step = random.randint(1,range)
print(range, step, msgLen, int(msgLen, 2))
i += step
random.seed(step)
step = random.randint(1,range)
while i < secret_len:
  # if secret_bytes[0][i] > 63:
  temp_msg += str(secret_bytes[0][i] & 1)
  if len(temp_msg) == 8:
    if chr(int(temp_msg, 2)) == "$":
      break
    ex_msg += chr(int(temp_msg, 2))
    temp_msg = ""
  i += step
  random.seed(step)
  step = random.randint(1,range)
print("Extracted message: ", len(ex_msg))

15888 8667 0000000001110000 112
Extracted message:  11


In [9]:
ex_msg

'hello world'